In [1]:
import pandas as pd
import ast
import os
from pprint import pprint
import matplotlib.pyplot as plt
from auxiliar_func import *
from plot_func import *

# Importing the models
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

In [2]:
models = {
    'nb': GaussianNB,
    'lda': LDA,
    'qda': QDA,
    'knn': KNN,
    'logreg': LogisticRegression,
    'svm': LinearSVC,
    'rf': RandomForestClassifier,
    'xgb': XGBClassifier,
    'catboost': CatBoostClassifier,
}

df_tr = pd.read_csv('../train.csv')

# for xgboost
df_tr = df_tr.applymap(lambda x: x.replace('[', '').replace(']', '').replace('<', '') if isinstance(x, str) else x)

TARGET_METRIC = 'f1_macro'
SEED = 42
CV_FOLDS = 20

results = pd.DataFrame(columns=['model', 'accuracy', 'f1_macro', 'precision_macro', 'recall_macro'], dtype=float)
results.set_index('model', inplace=True)


for file in os.listdir('./results'):
    if file.endswith('.csv') and file.startswith('results_'):
        res = read_results('./results/'+file)
        prep_par, model_par = get_best_params('./results/'+file)
        mod_name = file.split('_')[1][:-4]
        model = models[mod_name](**model_par)
        score, y_pred, y_true = cross_validation(model, df_tr, prep_par, cv=CV_FOLDS, random_state=SEED, return_predict=True)

        results.loc[mod_name] = pd.Series(score)
        
        plot_conf_matrix(y_true, y_pred, './figures/conf_matrix_'+mod_name+'.pdf', show=False)
        
        

results = results.sort_values(by=TARGET_METRIC, ascending=False)

In [3]:
results.to_csv('./results/cv20_results.csv')
results.head(10)

,accuracy,f1_macro,precision_macro,recall_macro
model,,,,
catboost,0.953940,0.800256,0.807352,0.793767
xgb,0.952580,0.796274,0.800346,0.792595
rf,0.947503,0.780380,0.776899,0.784150
logreg,0.944990,0.774382,0.766083,0.783646
svm,0.945305,0.773174,0.767621,0.779223
lda,0.944969,0.764627,0.767230,0.762267
knn,0.945907,0.758366,0.773483,0.745369
qda,0.924985,0.741566,0.706241,0.799326
nb,0.903713,0.695970,0.660912,0.767947
